# Muse Data Elaborator

#### TODO:
1. (done) Data Recording
    1. (done) Choose the App
    1. (done) Download the data
1. Data Cleaning
    1. (done) Remove noise
    1. Remove invalid parts: sensor does not work -> remove that line
1. Feature Extraction
    1. (done) Calculation of features (five brain wave frequency bands)
    1. (done) Normalize data using the baseline.
1. "Machine Learning"
    1. Split data. (window of even 2 seconds; 5-60 seconds)
    1. Calculate the stress level.
    1. Label data.
1. (done) Data Visualization

#### TODO from notes:
1. windows of 10 seconds each; mean of the values
1. scenario ranking per participant (stress level)
1. sum the number of blinks and jaws, in experiment and in baseline
1. scenario ranking per participant (blinks and jaws)

1. restructure all the texts in the correct way

## Notes

##### Waves information
1. Alpha:
    - increases when a person is relaxed (the lower the busier)
    - blocked in task engagement
    
    
1. Beta:
    - increases in task engagement
    
    
1. Theta:
    - increases in demand and working memory load
    - sensitive to data difficulty
    - suppressed in task engagement
    
    
1. Delta:
    - sensitive to data difficulty

In [ ]:
import numpy as np
import pandas as pd
from bokeh.plotting import figure, show, output_file, save, curdoc
from bokeh.models import ColumnDataSource, HoverTool, NumeralTickFormatter, Title
from bokeh.models.widgets import Select
from bokeh.layouts import column, row, gridplot
from datetime import datetime as dt
from math import pi, sqrt
import os

# Scenarios Titles
experiment_labels = ['scenario 1', 'scenario 2', 'scenario 3']
all_scenario_labels = ['fishes 1', 'scenario 1', 'fishes 2', 'scenario 2', 'fishes 3', 'scenario 3']

# Ranges for plots
min_range = 200
max_range = -100

# Setting the threshold to have a good sensors signal - 4 HSI possible values: 1=Good, 2=Medium, 4=Bad
hsi_threshold = 8

In [ ]:
# 0. Calls all the algorithms in the right order
def execute_class(participant_no):
    
    # Read the file
    df = read_file(participant_no)
    
    # Clean the data and split it based on content
    df_records, df_elements, df_markers = clean_data(df)

    # Checking if the data returned is valid
    if str(type(df_records)) != "<class 'str'>":
        
        # Calculating the frequencies using the correct range
        df_prepared = extract_features(df_records)
        #visualize_simple_plot([df_prepared], "initial", ['All'])
        
        # Split the data into baseline data and experiment data
        sections = split_data(df_prepared, df_markers)
        #visualize_simple_plot(sections, "sections", all_scenario_labels)
        
        # Normalize the experiment based on its baseline and the quality of data
        normalized_sections = normalize_data(sections)
        #visualize_simple_plot(normalized_sections, "after_baseline_quality", experiment_labels)
        
        # Calculating the stress indicators
        stress_sections = calculate_stress(normalized_sections)
        visualize_complex_plot(stress_sections, "stress_sections", experiment_labels)
        
        # Splitting data into comparable windows
        window_sections = split_in_windows(stress_sections)
        

In [ ]:
# 1. Load the dato into dataframe
def read_file(participant_no):
    __file__ = participant_no + '.csv'
    my_absolute_dirpath = os.path.abspath(os.path.dirname(__file__))
    file_path = my_absolute_dirpath+"\\aData\\"+__file__
    df = pd.read_csv(file_path, sep=",")
    return df

In [ ]:
# 2. Removes useless columns and checks the correctness
def clean_data(df):
    # Removing useless columns and formatting
    df['TimeStamp'] = pd.to_datetime(df['TimeStamp'], errors='coerce')
    df_cleaned = df.drop(columns=['Gyro_X', 'Gyro_Y', 'Gyro_Z', 'Accelerometer_X', 'Accelerometer_Y', 'Accelerometer_Z', 'AUX_RIGHT', 'Battery'])

    # Extracting the markers from the dataframe
    df_markers = df_cleaned[df_cleaned['Elements'].str.contains('/Marker/', na=False)]
    df_markers = df_markers.reset_index(drop=True)
    df_markers = df_markers[['TimeStamp', 'Elements']]
    
    # Checking if the markers are correct - Markers are 7: 1x "marker 5", 3x "marker 1", 3x "marker 2"
    markers_no = df_markers.shape[0]
    marker_five_no = df_markers[df_markers['Elements'] == '/Marker/5'].shape[0]
    marker_one_no = df_markers[df_markers['Elements'] == '/Marker/1'].shape[0]
    marker_two_no = df_markers[df_markers['Elements'] == '/Marker/1'].shape[0]
    if not (markers_no == 7 and marker_five_no == 1 and marker_one_no == 3 and marker_two_no == 3):
        print("Markers are not of the desired number or type")
        return '', '', ''
    
    # Deleting all records before the initial marker timestamp (time spent positioning the sensor)
    df_start_markers = df_markers[df_markers['Elements'] == '/Marker/5']
    timestamp_start = df_start_markers['TimeStamp'].iloc[-1]
    df_formatted = df_cleaned[df_cleaned['TimeStamp'] >= timestamp_start]
    
    # Removing marker 5 from all dataframes
    df_markers = df_markers.iloc[1:]
    df_formatted = df_formatted.iloc[1:]

    # Extracting the elements - markers, blinks, clinches
    df_elements = df_formatted[df_formatted['Elements'].str.contains('/', na=False)]
    df_elements = df_elements[['TimeStamp', 'Elements']]
    df_elements = df_elements.dropna()
    df_elements = df_elements.reset_index(drop=True)

    # Extracting the records with the HeadBandOn
    df_records = df_formatted[df_formatted['HeadBandOn'] == 1]
    df_records = df_records.drop(columns=['Elements'])
    df_records = df_records.reset_index(drop=True)
    
    return df_records, df_elements, df_markers

In [ ]:
# 3. Transforming and extracting data, including data split and normalization
def extract_features(df_records):
    
    # Settings to change the data range
    old_range_max = 3
    old_range_min = -3
    old_range = (old_range_max - old_range_min) 
    new_range_max = 200
    new_range_min = -100
    new_range = (new_range_max - new_range_min)

    # Calculating Average Absolute Brain Waves
    df_avg_bw = df_records.drop(columns=['Delta_TP9', 'Delta_AF7', 'Delta_AF8', 'Delta_TP10', 'Theta_TP9', 'Theta_AF7', 'Theta_AF8', 'Theta_TP10', 'Alpha_TP9', 'Alpha_AF7', 'Alpha_AF8', 'Alpha_TP10', 'Beta_TP9', 'Beta_AF7', 'Beta_AF8', 'Beta_TP10', 'Gamma_TP9', 'Gamma_AF7', 'Gamma_AF8', 'Gamma_TP10'])
    old_alpha_value = (df_records['Alpha_TP9'] + df_records['Alpha_AF7'] + df_records['Alpha_AF8'] + df_records['Alpha_TP10'])/4
    old_beta_value = (df_records['Beta_TP9'] + df_records['Beta_AF7'] + df_records['Beta_AF8'] + df_records['Beta_TP10'])/4
    old_delta_value = (df_records['Delta_TP9'] + df_records['Delta_AF7'] + df_records['Delta_AF8'] + df_records['Delta_TP10'])/4
    old_gamma_value = (df_records['Gamma_TP9'] + df_records['Gamma_AF7'] + df_records['Gamma_AF8'] + df_records['Gamma_TP10'])/4
    old_theta_value = (df_records['Theta_TP9'] + df_records['Theta_AF7'] + df_records['Theta_AF8'] + df_records['Theta_TP10'])/4

    # Converting in new range
    df_avg_bw['Alpha_Avg'] = (((old_alpha_value - old_range_min) * new_range) / old_range) + new_range_min
    df_avg_bw['Beta_Avg'] = (((old_beta_value - old_range_min) * new_range) / old_range) + new_range_min
    df_avg_bw['Delta_Avg'] = (((old_delta_value - old_range_min) * new_range) / old_range) + new_range_min
    df_avg_bw['Gamma_Avg'] = (((old_gamma_value - old_range_min) * new_range) / old_range) + new_range_min
    df_avg_bw['Theta_Avg'] = (((old_theta_value - old_range_min) * new_range) / old_range) + new_range_min
    
    return df_avg_bw


In [ ]:
# 4. Splitting data based on markers
def split_data(df, df_markers):
    
    # Setting initial variable
    i = 0
    sections = []
    
    # Splitting the data and visualizing them separately
    prev_timestamp = "null"
    markers_timestamps = df_markers['TimeStamp'].tolist()
    for timestamp in markers_timestamps:
        
        # Taking different actions for the dataframe splitting
        if prev_timestamp != "null":
            section = df[
                df['TimeStamp'] > prev_timestamp
            ]
            section = section[
                section['TimeStamp'] <= timestamp
            ]
            
            sections.append(section)
            i=i+1
            
        prev_timestamp = timestamp

    # Visualizing the last part of the experiment
    section = df[
        df['TimeStamp'] > prev_timestamp
    ]
    sections.append(section)
    
    # Returns 6 sections: (baseline + scenario) x3
    return sections

In [ ]:
# 5. Normalizing the data based on the baseline
def normalize_data(sections):
    
    # Min and Max range for plot
    global min_range
    global max_range
        
    # Retriving threshold for quality of data
    global hsi_threshold
    
    # Declaring variables for the method execution
    total_lines = 0
    total_usable_lines = 0
    frequency_columns = ['Alpha_Avg', 'Beta_Avg', 'Delta_Avg', 'Gamma_Avg', 'Theta_Avg']
    
    # Declaring returning variables
    new_sections = []
    
    # Iterating through the 3 baseline-experiment sections
    for i in range(0,6,2):
            
        # Baseline - Extracting the last minute of recording 
        df_baseline = sections[i]
        baseline_last_timestamp = df_baseline.iloc[-1]['TimeStamp']
        baseline_start = baseline_last_timestamp - pd.Timedelta(minutes=1)        
        df_baseline_min = df_baseline[df_baseline['TimeStamp'] >= baseline_start]
        
        # Baseline - Removing low quality data
        df_baseline_min['Sensor_Quality'] = df_baseline_min['HSI_TP9'] + df_baseline_min['HSI_AF7'] + df_baseline_min['HSI_AF8'] + df_baseline_min['HSI_TP10']
        df_baseline_min_cleaned = df_baseline_min[df_baseline_min['Sensor_Quality'] <= hsi_threshold]
        
        # Baseline - Calculating frequencies average as baseline
        baseline_frequencies = []
        print("Frequency Mean for scenario " +str(get_scenario_no(i))+":")
        for j in range(0,5):
            baseline_frequencies.append(df_baseline_min_cleaned[frequency_columns[j]].mean())
            print("-- " + frequency_columns[j] + ": " +('%.2f' % (baseline_frequencies[j],)).rstrip('0').rstrip('.'))
        
        # Experiment - Removing low quality data
        df_experiment = sections[i+1]
        df_experiment['Sensor_Quality'] = df_experiment['HSI_TP9'] + df_experiment['HSI_AF7'] + df_experiment['HSI_AF8'] + df_experiment['HSI_TP10']
        df_experiment_cleaned = df_experiment[df_experiment['Sensor_Quality'] <= hsi_threshold]
        
        # Experiment - Defining the data quality
        lines = df_experiment.shape[0]
        usable_lines = df_experiment_cleaned.shape[0]
        total_lines = total_lines + lines
        total_usable_lines = total_usable_lines + usable_lines
        print(str(usable_lines) + "/" + str(lines) + " usable lines for scenario " +str(get_scenario_no(i)))
        print()
        
        # Removing baseline from experiment
        df_experiment_baseline = df_experiment_cleaned.copy()
        for k in range(0,5):
        #    df_experiment_baseline[frequency_columns[k]] = df_experiment_cleaned[frequency_columns[k]] - baseline_frequencies[k]
            df_experiment_baseline[frequency_columns[k]] = df_experiment_cleaned[frequency_columns[k]]
            
        # Storing the min and max range of plot
        for wave in frequency_columns:
            this_min = min(df_experiment_baseline[wave])
            this_max = max(df_experiment_cleaned[wave])
            if min_range > this_min:
                min_range = this_min
            if max_range < this_max:
                max_range = this_max
        
        # Storing the elaborated sections
        new_sections.append(df_experiment_baseline)
        
    # Printing the survey data quality
    print("Total usable lines for this participant is "+('%.2f' % (100*total_usable_lines/total_lines,)).rstrip('0').rstrip('.')+
          "% - threshold: "+str(hsi_threshold))

    return new_sections

# Returns the scenario number based on the iterator of method "normalize_data"
def get_scenario_no(i):
    if i == 0:
        return 1
    elif i == 2:
        return 2
    else:
        return 3

In [ ]:
# 6. Define stress levels
def calculate_stress(sections):
    
    # Min and Max range for plot
    global min_range
    global max_range
    
    for section_df in sections:
        
        # Calculating first ratio: Beta / (Alpha + Theta) --> task difficulty indicator + task engagement
        section_df['First_Ratio'] = section_df['Beta_Avg'] / (section_df['Alpha_Avg'] + section_df['Theta_Avg'])
        
        # Calculating second ratio: Theta / (Alpha + Beta) --> task difficulty indicator
        section_df['Second_Ratio'] = section_df['Theta_Avg'] / (section_df['Alpha_Avg'] + section_df['Beta_Avg'])
        
        # Calculating the max-min range for plot
        for ratio in ['First_Ratio', 'Second_Ratio']:
            this_min = min(section_df[ratio])
            this_max = max(section_df[ratio])
            if min_range > this_min:
                min_range = this_min
            if max_range < this_max:
                max_range = this_max
        
    return sections

In [ ]:





# 7. Create windows
def split_in_windows(sections):
    print()
    
    
    
    
    

In [ ]:
# END: Visualizing the data for the given simple sections
def visualize_simple_plot(sections, extra_title, labels):
    
    # Setting initial variable
    i = 0
    
    # Visualizing all the sections
    for section in sections:
        
        # Defining variables used in method
        df = section
        title = labels[i]
        
        # Saving data in ColumnDataSource
        data = {'TimeStamp': np.array(df['TimeStamp'], dtype='i8').view('datetime64[ms]').tolist(),
                'Alpha': list(df['Alpha_Avg']),
                'Beta': list(df['Beta_Avg']),
                'Delta': list(df['Delta_Avg']),
                'Gamma': list(df['Gamma_Avg']), 
                'Theta': list(df['Theta_Avg']),
                'TimeStamp_tooltip': [x.strftime("%Y-%m-%d %H:%M:%S") for x in df['TimeStamp']]
                }
        source = ColumnDataSource(data=data)

        # Calculating the graph ranges
        global min_range
        global max_range
        smallest = min_range
        largest = max_range
        smallest -= 5
        largest += 5

        # Plotting the data
        p = figure(x_range=(min(data['TimeStamp']), max(data['TimeStamp'])), y_range=(smallest, largest), plot_width=1500, plot_height=600, title=extra_title+": Plot of "+title)
        p.line(x='TimeStamp', y='Alpha', source=source, color="#3DB3FE", line_width=2, legend=dict(value="Alpha"))
        p.line(x='TimeStamp', y='Beta', source=source, color="#38A967", line_width=2, legend=dict(value="Beta"))
        p.line(x='TimeStamp', y='Delta', source=source, color="#C93030", line_width=2, legend=dict(value="Delta"))
        p.line(x='TimeStamp', y='Gamma', source=source, color="#F1A219", line_width=2, legend=dict(value="Gamma"))
        p.line(x='TimeStamp', y='Theta', source=source, color="#A822F3", line_width=2, legend=dict(value="Theta"))

        # Adding hover and other visualization tools
        hover = HoverTool()
        hover.tooltips=[
            ("Value", "$y"),
            ("Timestamp", "@TimeStamp_tooltip")
        ]
        p.add_tools(hover)
        p.add_layout(Title(text="TimeStamp", align="center"), "below")
        p.add_layout(Title(text="Frequency", align="center"), "left")
        p.xaxis.major_label_orientation = np.pi / 4
        p.legend.location = 'top_right'
        
        show(p)
        i = i + 1
    

In [ ]:
# END: Visualizing the data for the given complex sections
def visualize_complex_plot(sections, extra_title, labels):
    
    # Setting initial variable
    i = 0
    
    # Visualizing all the sections
    for section in sections:
        
        # Defining variables used in method
        df = section
        title = labels[i]
        
        # Saving data in ColumnDataSource
        data = {'TimeStamp': np.array(df['TimeStamp'], dtype='i8').view('datetime64[ms]').tolist(),
                'First_Ratio': list(df['First_Ratio']),
                'Second_Ratio': list(df['Second_Ratio']),
                'TimeStamp_tooltip': [x.strftime("%Y-%m-%d %H:%M:%S") for x in df['TimeStamp']],
                'Stress_Level': list (df['First_Ratio'] + df['Second_Ratio'])
                }
        source = ColumnDataSource(data=data)
        
        # Figures configuration
        x_min = min(data['TimeStamp'])
        x_max = max(data['TimeStamp'])
        p_smallest = 0.2
        p_largest = 0.8
        p2_smallest = 0.4
        p2_largest = 1.6
        plot_width = 1500
        plot_height = 400

        # Plotting the data
        p = figure(x_range=(x_min, x_max), y_range=(p_smallest, p_largest), plot_width=plot_width, plot_height=plot_height, title=extra_title+": Plot of "+title)
        p.line(x='TimeStamp', y='First_Ratio', source=source, color="#f1df19", line_width=2, legend=dict(value="First_Ratio"))
        p.line(x='TimeStamp', y='Second_Ratio', source=source, color="#f15319", line_width=2, legend=dict(value="Second_Ratio"))
        
        # Adding hover and other visualization tools
        hover = HoverTool()
        hover.tooltips=[
            ("Value", "$y"),
            ("Timestamp", "@TimeStamp_tooltip")
        ]
        p.add_tools(hover)
        p.add_layout(Title(text="TimeStamp", align="center"), "below")
        p.add_layout(Title(text="Frequency", align="center"), "left")
        p.xaxis.major_label_orientation = np.pi / 4
        p.legend.location = 'top_right'
        
        # Plotting the data
        p2 = figure(x_range=(x_min, x_max), y_range=(p2_smallest, p2_largest), plot_width=plot_width, plot_height=plot_height, title=extra_title+": Stress level of "+title)
        p2.line(x='TimeStamp', y='Stress_Level', source=source, color="green", line_width=2, legend=dict(value="Stress_Level"))
        
        # Visualize the plots and increment the iterator
        show(column(p, p2))
        i = i + 1

In [ ]:
# -------------------Main Method-------------------------------------------

# TODO change it so that the algorithm takes this as input
participants = ['02']
#participants = ['01', '02']

for participant in participants:
    print("===== Participant "+participant+" =====")
    execute_class(participant)
    print()